In [1]:
import cv2
import os
from tqdm import tqdm

def extract_frames(video_path, output_dir, frame_rate=1, crop=None, start_time=0, end_time=1):
    """
    Extract frames from a video between specified start and end times at a given frame rate.
    Optionally, crop frames based on a provided crop area.
    
    :param video_path: Path to the video file
    :param output_dir: Directory where extracted frames will be saved
    :param frame_rate: The frame rate (how often to extract frames in seconds)
    :param crop: Optional tuple (x1, y1, x2, y2) defining the region to crop
    :param start_time: Time in seconds to start extracting frames (inclusive)
    :param end_time: Time in seconds to stop extracting frames (exclusive)
    """
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file {video_path}")
        return

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)  # Frames per second
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Total frames in the video
    video_duration = total_frames / fps  # Duration in seconds
    interval = int(fps / frame_rate)  # Interval between frames based on desired frame rate
    
    # Validate start_time and end_time
    if start_time < 0:
        print("Error: start_time must be >= 0.")
        return
    if end_time > video_duration:
        print(f"Error: end_time must be <= video duration ({video_duration:.2f} seconds).")
        return
    if start_time >= end_time:
        print("Error: start_time must be less than end_time.")
        return

    # Set the starting point for extraction
    cap.set(cv2.CAP_PROP_POS_MSEC, start_time * 1000)  # Convert start_time to milliseconds

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    frame_count = 0
    extracted_count = 0

    total_extractable_frames = int((end_time - start_time) * frame_rate)  # Calculate the total frames between start and end times

    # Show progress bar using tqdm
    with tqdm(total=total_extractable_frames+1, desc="Extracting frames", ncols=100, bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed} < {remaining}, {rate_fmt}]") as pbar:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                continue

            # Get current time in seconds
            current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000  # Convert milliseconds to seconds

            # If the current time is beyond the end time, stop the extraction
            if current_time >= end_time:
                break
            
            # If it's time to extract a frame, crop it if needed
            if frame_count % interval == 0 and current_time >= start_time:
                if crop:
                    x1, y1, x2, y2 = crop
                    frame = frame[y1:y2, x1:x2]  # Crop the frame
                
                # Save the frame as PNG
                frame_path = os.path.join(output_dir, f"frame_{extracted_count:05d}.png")
                cv2.imwrite(frame_path, frame)
                extracted_count += 1
                pbar.update(1)  # Update the progress bar by one step
            frame_count += 1

    cap.release()
    
    print(f"Extracted {extracted_count} frames from {video_path} between {start_time} and {end_time or video_duration} seconds.")

# List of video files to process
video_files = ['experiment.mp4']
output_dir = 'experiment'

# Define crop area as (x1, y1, x2, y2) in pixels (optional)
crop_area = None  # Example crop area (you can set it to None to skip cropping)

# Define start and end times for frame extraction (in seconds)
start_time = 0 # Start extracting frames from start_time (in seconds)
end_time = 15 # Stop extracting frames at end_time (in seconds)
frame_rate = 1 # Rate at which the frames will be extracted

# Extract frames at a rate of 2 seconds per frame with optional cropping
for video in video_files:
    extract_frames(video, output_dir, frame_rate=frame_rate, crop=crop_area, start_time=start_time, end_time=end_time)

Extracting frames:  94%|███████████████████████████████████████▍  | 15/16 [00:15 < 00:01,  1.06s/it]


KeyboardInterrupt: 